# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7ff15f3f7fa0>
├── 'a' --> tensor([[-1.0982, -1.0263, -1.1308],
│                   [ 0.1781,  0.5325, -0.3560]])
└── 'x' --> <FastTreeValue 0x7ff15f3f7b80>
    └── 'c' --> tensor([[ 0.2030, -1.2996,  1.4354,  0.0244],
                        [-0.6007, -1.3824,  0.8323,  0.8619],
                        [-1.0431,  1.1787,  0.0660, -1.4907]])

In [4]:
t.a

tensor([[-1.0982, -1.0263, -1.1308],
        [ 0.1781,  0.5325, -0.3560]])

In [5]:
%timeit t.a

64.9 ns ± 0.031 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7ff15f3f7fa0>
├── 'a' --> tensor([[ 0.3097, -0.8617,  0.2596],
│                   [ 0.4186,  0.2571, -0.5379]])
└── 'x' --> <FastTreeValue 0x7ff15f3f7b80>
    └── 'c' --> tensor([[ 0.2030, -1.2996,  1.4354,  0.0244],
                        [-0.6007, -1.3824,  0.8323,  0.8619],
                        [-1.0431,  1.1787,  0.0660, -1.4907]])

In [7]:
%timeit t.a = new_value

62 ns ± 0.0678 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.0982, -1.0263, -1.1308],
               [ 0.1781,  0.5325, -0.3560]]),
    x: Batch(
           c: tensor([[ 0.2030, -1.2996,  1.4354,  0.0244],
                      [-0.6007, -1.3824,  0.8323,  0.8619],
                      [-1.0431,  1.1787,  0.0660, -1.4907]]),
       ),
)

In [10]:
b.a

tensor([[-1.0982, -1.0263, -1.1308],
        [ 0.1781,  0.5325, -0.3560]])

In [11]:
%timeit b.a

51.3 ns ± 0.0392 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.1528,  0.4901, -1.2728],
               [ 0.7268, -0.6593, -0.1476]]),
    x: Batch(
           c: tensor([[ 0.2030, -1.2996,  1.4354,  0.0244],
                      [-0.6007, -1.3824,  0.8323,  0.8619],
                      [-1.0431,  1.1787,  0.0660, -1.4907]]),
       ),
)

In [13]:
%timeit b.a = new_value

477 ns ± 0.151 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

847 ns ± 0.404 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 18.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

138 µs ± 448 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

140 µs ± 542 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7ff15c36f460>
├── 'a' --> tensor([[[-1.0982, -1.0263, -1.1308],
│                    [ 0.1781,  0.5325, -0.3560]],
│           
│                   [[-1.0982, -1.0263, -1.1308],
│                    [ 0.1781,  0.5325, -0.3560]],
│           
│                   [[-1.0982, -1.0263, -1.1308],
│                    [ 0.1781,  0.5325, -0.3560]],
│           
│                   [[-1.0982, -1.0263, -1.1308],
│                    [ 0.1781,  0.5325, -0.3560]],
│           
│                   [[-1.0982, -1.0263, -1.1308],
│                    [ 0.1781,  0.5325, -0.3560]],
│           
│                   [[-1.0982, -1.0263, -1.1308],
│                    [ 0.1781,  0.5325, -0.3560]],
│           
│                   [[-1.0982, -1.0263, -1.1308],
│                    [ 0.1781,  0.5325, -0.3560]],
│           
│                   [[-1.0982, -1.0263, -1.1308],
│                    [ 0.1781,  0.5325, -0.3560]]])
└── 'x' --> <FastTreeValue 0x7ff0874dc040>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33.1 µs ± 61.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7ff15f3f7160>
├── 'a' --> tensor([[-1.0982, -1.0263, -1.1308],
│                   [ 0.1781,  0.5325, -0.3560],
│                   [-1.0982, -1.0263, -1.1308],
│                   [ 0.1781,  0.5325, -0.3560],
│                   [-1.0982, -1.0263, -1.1308],
│                   [ 0.1781,  0.5325, -0.3560],
│                   [-1.0982, -1.0263, -1.1308],
│                   [ 0.1781,  0.5325, -0.3560],
│                   [-1.0982, -1.0263, -1.1308],
│                   [ 0.1781,  0.5325, -0.3560],
│                   [-1.0982, -1.0263, -1.1308],
│                   [ 0.1781,  0.5325, -0.3560],
│                   [-1.0982, -1.0263, -1.1308],
│                   [ 0.1781,  0.5325, -0.3560],
│                   [-1.0982, -1.0263, -1.1308],
│                   [ 0.1781,  0.5325, -0.3560]])
└── 'x' --> <FastTreeValue 0x7ff15f3f7b50>
    └── 'c' --> tensor([[ 0.2030, -1.2996,  1.4354,  0.0244],
                        [-0.6007, -1.3824,  0.8323,  0.8619],
                 

In [23]:
%timeit t_cat(trees)

30.8 µs ± 34.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

67.9 µs ± 203 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.2030, -1.2996,  1.4354,  0.0244],
                       [-0.6007, -1.3824,  0.8323,  0.8619],
                       [-1.0431,  1.1787,  0.0660, -1.4907]],
              
                      [[ 0.2030, -1.2996,  1.4354,  0.0244],
                       [-0.6007, -1.3824,  0.8323,  0.8619],
                       [-1.0431,  1.1787,  0.0660, -1.4907]],
              
                      [[ 0.2030, -1.2996,  1.4354,  0.0244],
                       [-0.6007, -1.3824,  0.8323,  0.8619],
                       [-1.0431,  1.1787,  0.0660, -1.4907]],
              
                      [[ 0.2030, -1.2996,  1.4354,  0.0244],
                       [-0.6007, -1.3824,  0.8323,  0.8619],
                       [-1.0431,  1.1787,  0.0660, -1.4907]],
              
                      [[ 0.2030, -1.2996,  1.4354,  0.0244],
                       [-0.6007, -1.3824,  0.8323,  0.8619],
                       [-1.0431,  1.1787,  0.0660, -1.4907]],

In [26]:
%timeit Batch.stack(batches)

78.7 µs ± 59.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.2030, -1.2996,  1.4354,  0.0244],
                      [-0.6007, -1.3824,  0.8323,  0.8619],
                      [-1.0431,  1.1787,  0.0660, -1.4907],
                      [ 0.2030, -1.2996,  1.4354,  0.0244],
                      [-0.6007, -1.3824,  0.8323,  0.8619],
                      [-1.0431,  1.1787,  0.0660, -1.4907],
                      [ 0.2030, -1.2996,  1.4354,  0.0244],
                      [-0.6007, -1.3824,  0.8323,  0.8619],
                      [-1.0431,  1.1787,  0.0660, -1.4907],
                      [ 0.2030, -1.2996,  1.4354,  0.0244],
                      [-0.6007, -1.3824,  0.8323,  0.8619],
                      [-1.0431,  1.1787,  0.0660, -1.4907],
                      [ 0.2030, -1.2996,  1.4354,  0.0244],
                      [-0.6007, -1.3824,  0.8323,  0.8619],
                      [-1.0431,  1.1787,  0.0660, -1.4907],
                      [ 0.2030, -1.2996,  1.4354,  0.0244],
                   

In [28]:
%timeit Batch.cat(batches)

147 µs ± 161 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

324 µs ± 1.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
